In [1]:
import math
import json
import numpy as np
import falconn
import timeit
from __future__ import print_function
import psycopg2
import pandas as pd
from configparser import ConfigParser
import requests
import re
import traceback

In [50]:
def config(filename='prepare_data.ini', section='phonetic'):
    parser = ConfigParser()
    parser.read(filename)
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
 
    return db

def db_connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    params = config()
    conn = psycopg2.connect(**params)
    return conn

def read_dataframe():
    conn = db_connect()
    result = None
    try:
        query = """
            SELECT * FROM validation.validation_data
            """
        result = pd.read_sql(query, con=conn, index_col='id')
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return result

def update_db_row(idx, status, login, eng_invalidated, arb_invalidated, arb_yamli_invalidated):
    conn = db_connect()
    try:
        sql = """
            UPDATE validation.validation_data 
            SET status = %s, login = %s, eng_invalidated = %s, arb_invalidated = %s, arb_yamli_invalidated = %s
            WHERE id = %s
            """
        cur = conn.cursor()
        cur.execute(sql, (status, login, eng_invalidated, arb_invalidated, arb_yamli_invalidated, idx))
        conn.commit()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    
    
def claim_db_row(idx, login):
    conn = db_connect()
    try:
        sql = """
            UPDATE validation.validation_data 
            SET status = %s, login = %s 
            WHERE id = %s
            """
        cur = conn.cursor()
        cur.execute(sql, (int(1), login, int(idx)))
        conn.commit()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    
def load_next_row(login):
    result = dict()
    row = None
    
    x = df[df.status == 0]
    if len(x) > 0:
        row = x.iloc[0]
    if row is None:
        return None
    
    idx = int(row.name)
    
    df.set_value(col='status', index=idx, value=1)
    df.set_value(col='login', index=idx, value=login)
    row['status'] = int(1)
    row['login'] = login
    
    result['id'] = idx
    result['eng'] = row['eng']
    result['arb'] = row['arb']
    result['eng_variants'] = {name : 0 for name in row['eng_variants'].split(",")}
    result['arb_variants'] = {name : 0 for name in row['arb_variants'].split(",")}
    result['arb_yamli_variants'] = {name : 0 for name in row['arb_yamli_variants'].split(",")}
    
    claim_db_row(idx=idx, login=login)
    
    return result

def validate_row(idx, data, login):
    
    df.set_value(col='status', index=idx, value=2)
    df.set_value(col='login', index=idx, value=login)
    
    arb_invalidated = data['arb_invalidated']
    arb_yamli_invalidated = data['arb_yamli_invalidated']
    eng_invalidated = data['eng_invalidated']
    arb = ''
    arb_yamli = ''
    eng = ''
    
    for x in arb_invalidated:
        arb = arb + ',' + x
    arb = arb[1:]
    
    for x in arb_yamli_invalidated:
        arb_yamli = arb_yamli + ',' + x
    arb_yamli = arb_yamli[1:]
    
    for x in eng_invalidated:
        eng = eng + ',' + x
    eng = eng[1:]
    
    df.set_value(col='arb_invalidated', index=idx, value=arb)
    df.set_value(col='arb_yamli_invalidated', index=idx, value=arb_yamli)
    df.set_value(col='eng_invalidated', index=idx, value=eng)
    
    update_db_row(idx=int(idx), login=login, status=2, arb_invalidated=arb, eng_invalidated=eng, arb_yamli_invalidated=arb_yamli)


In [3]:
df = read_dataframe()

In [ ]:
'''
row = load_next_row(login)

updated = dict()
updated['arb_invalidated'] = {}
updated['arb_yamli_invalidated'] = {}
updated['eng_invalidated'] = {}

i = 0
for a in row['arb_variants']:
    updated['arb_invalidated'][a] = i%2 
    i = i + 1
i = 0
for a in row['arb_yamli_variants']:
    updated['arb_yamli_invalidated'][a] = i%2 
    i = i + 1
i = 0
for a in row['eng_variants']:
    updated['eng_invalidated'][a] = i%2 
    i = i + 1

validate_row(row['id'], updated, 'cez')
'''

In [53]:
# POST /validate

req = json.loads(REQUEST)
body = req['body']

login = 'unspecified'
index = -1
updated = None

if 'login' in body:
    login = body['login']
if 'index' in body and len(body['index']) > 0:
    index = body['index']
if 'updated' in body:
    updated = body['updated']

result = dict()
msg = ''

if (updated is not None and index != -1):
    validate_row(idx=index, data=updated, login=login)
    
result['data'] = load_next_row(login)
result['message'] = msg
    
print(json.dumps({'result': result}))

{'eng_invalidated': ['ZAID'], 'arb_invalidated': ['ذيد'], 'arb_yamli_invalidated': ['زيد']}
['ذيد']
['زيد']
['ZAID']
ذيد
زيد
ZAID
{"result": {"data": {"id": 38, "eng": "HISHAM", "arb": "\u0647\u0634\u0627\u0645", "eng_variants": {"HISHAM": 0, "HESHAM": 0, "HUSHAM": 0}, "arb_variants": {"\u062d\u0634\u0627\u0645": 0, "\u062d\u0627\u0634\u0645": 0, "\u0647\u0648\u0634\u0627\u0645": 0, "\u062d\u064a\u0634\u0645": 0, "\u062d\u0633\u0647\u0645": 0, "\u0647\u0648\u0634\u0645": 0, "\u062d\u062b\u0647\u0645": 0, "\u062d\u0648\u0634\u0645": 0, "\u0647\u0634\u0627\u0645": 0, "\u062d\u0627\u062b\u0647\u0645": 0, "\u062d\u0634\u0645": 0, "\u0647\u0648\u0633\u0647\u0645": 0, "\u0647\u064a\u0634\u0627\u0645": 0, "\u0647\u0627\u0634\u0645": 0, "\u0647\u064a\u0634\u0645": 0, "\u062d\u0627\u0633\u0647\u0645": 0, "\u062d\u064a\u0634\u0627\u0645": 0, "\u0647\u0634\u0645": 0}, "arb_yamli_variants": {"\u0647\u0634\u0627\u0645": 0, "\u0647\u064a\u0634\u0627\u0645": 0}}, "message": ""}}


/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [52]:
'''row = load_next_row('test')

updated = dict()
updated['arb_invalidated'] = {}
updated['arb_yamli_invalidated'] = {}
updated['eng_invalidated'] = {}

i = 0
for a in row['arb_variants']:
    updated['arb_invalidated'][a] = i%2 
    i = i + 1
i = 0
for a in row['arb_yamli_variants']:
    updated['arb_yamli_invalidated'][a] = i%2 
    i = i + 1
i = 0
for a in row['eng_variants']:
    updated['eng_invalidated'][a] = i%2 
    i = i + 1

b ={}
b['login'] = 'test'
b['index'] = row['id']
b['updated'] = updated

REQUEST = json.dumps({
'path' : {},
'args' : {},
'body' : b
})
REQUEST'''


REQUEST = '{"body":{"login":"dsaf","index":"106","updated":{"eng_invalidated":["ZAID"],"arb_invalidated":["ذيد"],"arb_yamli_invalidated":["زيد"]}}}'
REQUEST

'{"body":{"login":"dsaf","index":"106","updated":{"eng_invalidated":["ZAID"],"arb_invalidated":["ذيد"],"arb_yamli_invalidated":["زيد"]}}}'

In [54]:
REQUEST

'{"body":{"login":"dsaf","index":"106","updated":{"eng_invalidated":["ZAID"],"arb_invalidated":["ذيد"],"arb_yamli_invalidated":["زيد"]}}}'